In [3]:
import os
import pandas as pd
import csv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from stemming.porter2 import stem

In [6]:
genre_files = os.listdir('reviews_per_genre')
for genre_file in genre_files:
    if (genre_file != 'Thriller.csv'): continue # uncomment line to test one genre
    file_data = pd.read_csv('reviews_per_genre/' + genre_file)
    reviews = file_data['review']
file_data.head()
#print (str(len(reviews)))

,name,review
0,181 Jurassic Park 1993,year hollywood film jurassic park dinosaurs br...
1,181 Jurassic Park 1993,adapted book unique story destined incredible ...
2,181 Jurassic Park 1993,loved movie utter amazement brachiosaurus eat ...
3,181 Jurassic Park 1993,jurassic park american science fiction adventu...
4,181 Jurassic Park 1993,epic movie big screen jurassic park years idea...


In [5]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 40
no_top_words = 10

genre_files = os.listdir('reviews_per_genre')
for genre_file in genre_files:
    if (genre_file != 'Thriller.csv'): continue # uncomment line to test one genre
    file_data = pd.read_csv('reviews_per_genre/' + genre_file)
    reviews = file_data['review']
    reviews1 = pd.Series.tolist(reviews)
    reviews1 = [[stem(word) for word in sentence.split(" ")] for sentence in reviews]
    reviews = []
    for review in reviews1:
        reviews.append(' '.join(review))
    
    '''# CountVectorizer
    
    vect = CountVectorizer(min_df=2, stop_words="english")
    vect1 = vect.fit_transform(reviews)
    #pd.DataFrame(vect1.toarray(), index=reviews, columns=vect.get_feature_names()).head(10)
    print ("Vocabulary size: {}".format(len(vect.vocabulary_)))
    #print ("Vocabulary content:\n {}".format(vect.vocabulary_))
    
    # Bag of Words
    
    bag_of_words = vect.transform(reviews)
    #print ("bag_of_words: {}".format(repr(bag_of_words)))
    
    # Feature Extraction
    
    feature_names = vect.get_feature_names()
    #print("Number of features: {}".format(len(feature_names)))'''
    
    try:
        tfidf_vect = TfidfVectorizer(min_df=2, stop_words='english')
        tfidf_vect = tfidf_vect.fit(reviews)
        features = tfidf_vect.get_feature_names()
        indices = np.argsort(tfidf_vect.idf_)[::-1]
    except:
        continue
    
    top_n = 15
    top_features = [features[i] for i in indices[1:top_n]]
    print (top_features)
    
    tfidf = tfidf_vect.fit_transform(reviews)
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=top_n, stop_words='english')
    tf = tf_vectorizer.fit_transform(reviews)
    tf_feature_names = tf_vectorizer.get_feature_names()
    
    # Bag of Words
    
    bag_of_words = tf_vectorizer.transform(reviews)
    #print ("bag_of_words: {}".format(repr(bag_of_words)))
    
    # Run NMF
    nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
#     with open('NMF/genres/'+genre_file[:-4], 'w') as f:
#         for topic_idx, topic in enumerate(nmf.components_):
#             for i in topic.argsort()[:-no_top_words - 1:-1]:
#                 f.write(features[i]+'\n')
    
    def display_topics(model, feature_names, no_top_words):
        for topic_idx, topic in enumerate(model.components_):
            print ("Topic %d:" % (topic_idx))
            print (" ".join([feature_names[i]
                            for i in topic.argsort()[:-no_top_words - 1:-1]]))

    no_top_words = 10
    display_topics(nmf, features, no_top_words)
    display_topics(lda, tf_feature_names, no_top_words)
    # Run LDA
    #lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

['misnam', 'mirabil', 'mirac', 'mirimax', 'mirren', 'mirth', 'misbehav', 'misc', 'miscarriag', 'miscellan', 'misconduct', 'misshapen', 'minnesotta', 'misslead']
Topic 0:
film enjoy recommend excel brilliant fan love view amaz masterpiec
Topic 1:
batman bane nolan wayn gotham bruce catwoman bale trilog gordon
Topic 2:
movi love recommend amaz enjoy definit better theater plot favorit
Topic 3:
scorses depart nicholson damon dicaprio costello martin jack matt wahlberg
Topic 4:
tarantino dog reservoir pulp fiction quentin madsen keitel roth heist
Topic 5:
donni darko frank gyllenha jake rabbit kelli travel bunni teenag
Topic 6:
shark jaw spielberg brodi quint shaw water beach hooper dreyfuss
Topic 7:
willi sixth osment cole bruce haley sens joel shyamalan boy
Topic 8:
bourn ultimatum jason damon cia supremaci matt greengrass chase trilog
Topic 9:
leonard memento memori pearc wife nolan backward guy rememb term
Topic 10:
runner blade replic deckard ford scott harrison human ridley sci
Topic

NameError: name 'lda' is not defined

In [6]:
from collections import defaultdict
import re
review = ''
with open('input_review.txt', 'r') as f:
    review = f.read()

REPLACE_WITH_TWO_SPACES = "(\&)|(\%)|(\$)|(\@)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\*)|(–)|([0-9]+)|(<br\s*/><br\s*/>)|(\.)|(\-)|(\/)|(\s+)|(\n)"
REPLACE_WITH_SPACE = "(\s+)"
stopwords =["spoiler", "spoilers"]
with open('stopwords.txt', 'r') as stopwords_file:
    for line in stopwords_file:
        stop = line.replace('\n', '')
        stopwords.append(stop)
'''import nltk.corpus
for e in list(nltk.corpus.stopwords.words("english")):
    if e not in stopwords:
        print (e)'''
#print (stopwords)
REPLACE_WORDS = "( [a-z]{1,2} )|( [a-z]{20,} )" # 1-2 charactered words & >= 20 charactered words
FINAL = "[^a-z\s]"
#print (words_to_replace)

review_clean = review.lower()
review_clean = " " + review_clean + " "
review_clean = review_clean.replace("'", "")
review_clean = re.sub(REPLACE_WITH_TWO_SPACES, "  ", review_clean)
for stopword in stopwords:
    #STOP = " "+stopword.replace("'","")+" "
    review_clean = review_clean.replace(" " + stopword.replace("'","") + " ", "  ")
    #review_clean = re.sub(STOP, "  ", review_clean)
review_clean = re.sub(REPLACE_WORDS, "  ", review_clean)
#print (review_clean)
review_clean = re.sub(REPLACE_WITH_SPACE, " ", review_clean)
review_clean = re.sub(FINAL, "", review_clean)
review_clean = review_clean.strip()
if (review_clean == 'null' or review_clean == ''):
    print ('Review is null or full of stopwords')
    exit(0)
inp_dict = defaultdict(int)
for word in review_clean.split():
    inp_dict[stem(word)] += 1

reviews = [stem(word) for word in review.split(" ")]
genre_files = os.listdir('NMF/genres')
probs = {}
for genre_file in genre_files:
    d = defaultdict(int)
    with open('NMF/genres/'+genre_file, 'r') as f:
        genre_review = f.read().splitlines()
        for r in genre_review:
            d[r] += 1
    bayesian_prob = 1
    for key in inp_dict:
        try:
            value = inp_dict[key]
            val = d[key]
            if (value * val > 0):
                bayesian_prob *= (value * val + 0.0)
        except:
            continue
    probs[genre_file] = bayesian_prob
sorted_keys = sorted(probs, key=probs.get, reverse=True)
for key in sorted_keys:
    print (key + "\t\t" + str(probs[key]))
    
#Take top 5 genres for a review and recommend top 4 movies of each genre!! 
genre_list = []
count = 0
for key in sorted_keys:
    count+=1
    if(count==5):
        break
    genre_list.append(key)

recommended_movies = []
for genre in genre_list:
    movies = []
    genre_df = pd.read_csv('reviews_per_genre/'+genre+'.csv')
    movies = genre_df.name.unique()
    ct = 0
    for movie in movies:
        ct+=1
        if(ct==5):
            break
        recommended_movies.append(movie)
print(recommended_movies)
        

Horror		59440669655040.0
Sci-Fi		116095057920.0
Music		82556485632.0
Western		57330892800.0
War		41278242816.0
Mystery		17199267840.0
Thriller		12899450880.0
Action		5662310400.0
History		2579890176.0
Romance		1274019840.0
Film-Noir		1146617856.0
Adventure		716636160.0
Biography		716636160.0
Drama		530841600.0
Sport		509607936.0
Animation		382205952.0
Crime		335923200.0
Fantasy		199065600.0
Musical		82944000.0
Family		37748736.0
Comedy		552960.0
['052 Alien 1979', '033 Psycho 1960', '223 Diabolique 1955', '060 The Shining 1980', '031 Interstellar 2014', '181 Jurassic Park 1993', '207 Logan 2017', '109 Metropolis 1927', '039 The Pianist 2002', '226 La La Land 2016', '044 Whiplash 2014', '061 Django Unchained 2012', '106 For a Few Dollars More 1965', '035 Once Upon a Time in the West 1968', '009 The Good, the Bad and the Ugly 1966']


In [33]:
vect1 = vect1.asfptype()
lsa = TruncatedSVD(2, algorithm = 'arpack')
tfidf_lsa = lsa.fit_transform(vect1)
print (tfidf_lsa)

[[ 5.05491356 -1.31062149]
 [ 4.3276431  -1.31616716]
 [ 7.07766276 -2.32395652]
 ...
 [ 0.58175011  0.03963518]
 [ 4.27792354  4.15163211]
 [ 2.97190488 -0.01765916]]
